In [2]:
import pandas as pd
import os

In [3]:

output_relative_dir = '../../data/landing/PTV/'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

In [1]:
import os
import requests
import zipfile

url = "https://s3.ap-southeast-2.amazonaws.com/cl-isd-prd-datashare-s3-delivery/Order_BCTPT5.zip"
download_path = "../../data/landing/PTV/Initial_PTV_data/Order_BCTPT5.zip"
extract_to_path = "../../data/landing/PTV/Initial_PTV_data/"

os.makedirs(extract_to_path, exist_ok=True)

response = requests.get(url)
with open(download_path, 'wb') as file:
    file.write(response.content)

with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

os.remove(download_path)

print("successful downloaded and ")

文件已成功下载并解压到目标路径，ZIP文件已删除。
